In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression 
import time

import pandas as pd
import numpy as np
from copy import deepcopy
import os
import csv
from itertools import cycle
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")
# warnings.resetwarnings() # To change it back (optional)

import sys
sys.path.append("/nobackup/gogandhi/alt_means_sans_k/")

from scripts.similarity_scores import clustering_method_values, get_scores
from scripts.nets_and_embeddings import *

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

import sys
sys.path.append("/nobackup/gogandhi/alt_means_sans_k/notebooks/20240513_changing_binary_classifier/Logistic-Regression-From-Scratch-main/src")
from custom_logistic_regression import LogisticRegression as CustomLogisticRegression

from scripts.similarity_scores import louvain, calc_esim,find_knn_edges



def alt_means_ootb(emb, communities, device_name="cuda:0", num_neighbors=100): 
    # Here emb is just an array of datapoints, where each row corresponds to a d dimensional array corresponding to a point.
    # Community table is an array with community partitions, which will be converted 
    # below into a pandas dataframe, with 'community_id' column containing the array data.

    

    # Normalize the vector of each node to have unit length. This normalization improves clustering.
    X = np.einsum("ij,i->ij", emb, 1 / np.maximum(np.linalg.norm(emb, axis=1), 1e-24)) 
    #This makes our binary classifier from dot to cosine similarity
    X = emb.copy()

    # Clustering


    rpos, cpos, vpos = find_knn_edges(emb, num_neighbors=num_neighbors, device = device_name)
    cneg = np.random.choice(emb.shape[0], len(cpos))
    vneg = np.array(np.sum(emb[rpos, :] * emb[cneg, :], axis=1)).reshape(-1)
    
    x_input = np.concatenate([vpos, vneg]).reshape((-1, 1))
    y_predicted = np.concatenate([np.ones_like(vpos), np.zeros_like(vneg)])
    model = LogisticRegression()
    model.fit(
        x_input,
        y_predicted),
            
    w1, b0 = model.coef_[0, 0], -model.intercept_[0] 

    ##############
    bias_coef = np.linalg.norm(np.sum(emb,axis=0)/N)
    
    lr = CustomLogisticRegression()
    lr.fit(x_input, y_predicted, bias_coef, epochs=150)
    pred = lr.predict(x_test)
    weights, bias = lr.coef_()
    accuracy = accuracy_score(y_test, pred)
    print(accuracy)
    ##############
    
    kmeans = KMeans(n_clusters= len(set(communities)), random_state=0).fit(X)
    kmeans_esim = calc_esim(communities, kmeans.labels_)
    
    
    proposed_labels = louvain(emb, w1, b0, device = device_name)
    esim = calc_esim(communities, proposed_labels) 
        
    
    return proposed_labels, kmeans.labels_, esim, kmeans_esim 

In [5]:
# Get network and emb data to try on the custom model


N=10000
params = {
    "N": N,
    "k": 50,
    "maxk":  int(np.sqrt(10 * N)),
    "minc": 50,
    "maxc": int(np.ceil(np.sqrt(N * 10))),
    "tau": 3.0,
    "tau2": 1.0,
    "mu": 0.65,
    }
            
emb_params = {
    "method": "node2vec",
    "window_length": 10,
    "walk_length": 80,
    "num_walks": 10,
    "dim": 64,
}

net, community_table, seed = create_network(params)
emb = create_embedding(net, emb_params)

#result_run_mu = clustering_method_values(net, community_table, emb, score_keys, "cuda:3")

In [ ]:
#from data import x_train, x_test, y_train, y_test

# bias_coef = np.linalg.norm(np.sum(x,axis=0)/N)

# lr = CustomLogisticRegression()
# lr.fit(x_train, y_train, bias_coef, epochs=150)
# pred = lr.predict(x_test)
# weights, bias = lr.coef_()
# accuracy = accuracy_score(y_test, pred)
# print(accuracy)

# model = LogisticRegression(solver='newton-cg', max_iter=150)
# model.fit(x_train, y_train)
# pred2 = model.predict(x_test)
# accuracy2 = accuracy_score(y_test, pred2)
# print(accuracy2)

In [ ]:
# For restarting kernel

# import os
# os._exit(00)

In [6]:
len(emb)

10000

In [57]:
# Normalize the vector of each node to have unit length. This normalization improves clustering.
X = np.einsum("ij,i->ij", emb, 1 / np.maximum(np.linalg.norm(emb, axis=1), 1e-24)) 
#This makes our binary classifier from dot to cosine similarity
X = emb.copy()

# Clustering

rpos, cpos, vpos = find_knn_edges(emb, num_neighbors=100, device = "cuda:0")
cneg = np.random.choice(emb.shape[0], len(cpos))
vneg = np.array(np.sum(emb[rpos, :] * emb[cneg, :], axis=1)).reshape(-1)

bias_coef = np.linalg.norm(np.sum(emb,axis=0)/N)

x_input = np.concatenate([vpos, vneg]).reshape((-1, 1)) - bias_coef
y_predicted = np.concatenate([np.ones_like(vpos), np.zeros_like(vneg)])

                             
model = LogisticRegression(fit_intercept=True)
model.fit(
    x_input,
    y_predicted)
w0, b0 = model.coef_[0, 0], -model.intercept_[0] 

In [58]:
w0,b0 # OG

(1.1373434635269928, -0.0)

In [42]:
w1,b1 # added bias with intercept

(7.120677304859043, 25.9372287597817)

(7.097692992403809, 8.875957487682873)

In [43]:
w2,b2 # added bias without intercept

(0.14020269574223815, -0.0)

(1.1367683364344097, -0.0)

In [59]:
proposed_labels = louvain(emb, w0, b0, device = "cuda:0")
esim = calc_esim(community_table['community_id'], proposed_labels) 
print(esim)    

0.0


In [44]:
proposed_labels = louvain(emb, w1, b1, device = "cuda:0")
esim = calc_esim(community_table['community_id'], proposed_labels) 
print(esim)    

0.2066986484057361


In [45]:
proposed_labels = louvain(emb, w2, b2, device = "cuda:0")
esim = calc_esim(community_table['community_id'], proposed_labels) 
print(esim)    

0.0


In [ ]:
##############
# import sys
# sys.path.append("/nobackup/gogandhi/alt_means_sans_k/notebooks/20240513_changing_binary_classifier/Logistic-Regression-From-Scratch-main/src")
# from custom_logistic_regression import LogisticRegression as CustomLogisticRegression
import copy
import numpy as np
from sklearn.metrics import accuracy_score

class CustomLogisticRegression():
    def __init__(self):
        self.losses = []
        self.train_accuracies = []


    # We are modifying the function as:
    #    y = (1 + exp(w.x + x_c)))^-1
    def fit(self, x, y, bias_coef, epochs):
        #x = self._transform_x(x)
        #y = self._transform_y(y)

        self.weights = np.zeros(x.shape[1])
        self.bias = bias_coef

        for i in range(epochs):
            x_dot_weights = np.matmul(self.weights, x.transpose()) + self.bias
            pred = self._sigmoid(x_dot_weights)
            loss = self.compute_loss(y, pred)
            error_w, error_b = self.compute_gradients(x, y, pred)
            self.update_model_parameters(error_w, error_b)

            pred_to_class = [1 if p > 0.5 else 0 for p in pred]
            self.train_accuracies.append(accuracy_score(y, pred_to_class))
            self.losses.append(loss)

    def compute_loss(self, y_true, y_pred):
        # binary cross entropy
        y_zero_loss = y_true * np.log(y_pred + 1e-9)
        y_one_loss = (1-y_true) * np.log(1 - y_pred + 1e-9)
        return -np.mean(y_zero_loss + y_one_loss)

    def compute_gradients(self, x, y_true, y_pred):
        # derivative of binary cross entropy
        difference =  y_pred - y_true
        gradient_b = np.mean(difference)
        gradients_w = np.matmul(x.transpose(), difference)
        gradients_w = np.array([np.mean(grad) for grad in gradients_w])

        return gradients_w, gradient_b

    def update_model_parameters(self, error_w, error_b):
        self.weights = self.weights - 0.1 * error_w
        #self.bias = self.bias - 0.1 * error_b 
        # Our bias is a fixed average similarity of vectors, so we don't update it

    def predict(self, x):
        x_dot_weights = np.matmul(x, self.weights.transpose()) + self.bias
        probabilities = self._sigmoid(x_dot_weights)
        return [1 if p > 0.5 else 0 for p in probabilities]
    
    def coef_(self):
        return self.weights, self.bias

    def _sigmoid(self, x):
        return np.array([self._sigmoid_function(value) for value in x])

    def _sigmoid_function(self, x):
        if x >= 0:
            z = np.exp(-x)
            return 1 / (1 + z)
        else:
            z = np.exp(x)
            return z / (1 + z)

    def _transform_x(self, x):
        x = copy.deepcopy(x)
        return x.values

    def _transform_y(self, y):
        y = copy.deepcopy(y)
        return y.values.reshape(y.shape[0], 1)


bias_coef = np.linalg.norm(np.sum(emb,axis=0)/N)

lr = CustomLogisticRegression()
lr.fit(x_input, y_predicted, bias_coef, epochs=10)
weights, bias = lr.coef_()
##############

In [21]:
weights, bias

(array([102377.68488969]), 1.1957085215886312)

In [8]:
from data import x_train, x_test, y_train, y_test

np.shape(x_train),np.shape(x_input)

((455, 30), (2020000, 1))

In [12]:
x_train

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
68,9.029,17.33,58.79,250.5,0.10660,0.14130,0.31300,0.04375,0.2111,0.08046,...,10.310,22.65,65.50,324.7,0.14820,0.43650,1.25200,0.17500,0.4228,0.11750
181,21.090,26.57,142.70,1311.0,0.11410,0.28320,0.24870,0.14960,0.2395,0.07398,...,26.680,33.48,176.50,2089.0,0.14910,0.75840,0.67800,0.29030,0.4098,0.12840
63,9.173,13.86,59.20,260.9,0.07721,0.08751,0.05988,0.02180,0.2341,0.06963,...,10.010,19.23,65.59,310.1,0.09836,0.16780,0.13970,0.05087,0.3282,0.08490
248,10.650,25.22,68.01,347.0,0.09657,0.07234,0.02379,0.01615,0.1897,0.06329,...,12.250,35.19,77.98,455.7,0.14990,0.13980,0.11250,0.06136,0.3409,0.08147
60,10.170,14.88,64.55,311.9,0.11340,0.08061,0.01084,0.01290,0.2743,0.06960,...,11.020,17.45,69.86,368.6,0.12750,0.09866,0.02168,0.02579,0.3557,0.08020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,8.888,14.64,58.79,244.0,0.09783,0.15310,0.08606,0.02872,0.1902,0.08980,...,9.733,15.67,62.56,284.4,0.12070,0.24360,0.14340,0.04786,0.2254,0.10840
106,11.640,18.33,75.17,412.5,0.11420,0.10170,0.07070,0.03485,0.1801,0.06520,...,13.140,29.26,85.51,521.7,0.16880,0.26600,0.28730,0.12180,0.2806,0.09097
270,14.290,16.82,90.30,632.6,0.06429,0.02675,0.00725,0.00625,0.1508,0.05376,...,14.910,20.65,94.44,684.6,0.08567,0.05036,0.03866,0.03333,0.2458,0.06120
435,13.980,19.62,91.12,599.5,0.10600,0.11330,0.11260,0.06463,0.1669,0.06544,...,17.040,30.80,113.90,869.3,0.16130,0.35680,0.40690,0.18270,0.3179,0.10550


In [14]:
np.shape(emb)

(10000, 64)